# UQ MDS - Assessment administration

## Required
The cell below must always be run once on opening a new Jupyter notebook instance. The script will verify that the API token is valid and load all required objects.

In [ ]:
from ghca.gitlink import GitHubLink
ghl = GitHubLink('config/TEST1000_2018_S1.json')

## Setup student groups
Groups can be specified for an item of assessment. Run this to import the student groups from a CSV file.<br />
<br />
This must be run before preparing the assessment. If the assessment was specified to have `max-group-size = 1` (in `admin_course.ipynb`) then the input will be ignored. It will just create individual groups.

* Students which are a part of this course but not in the CSV file will be assumed to be doing the assessment by themselves.
* Creates `groups.json` in the `main-dir` (specified in `admin_course.ipynb`) of the assessment directory (located in the instructors repository).
* Each group name must be unique, this will form the name of the repository for the student group.

In [ ]:
IMPORT_STUDENT_GROUPS_CSV = """GroupName|Members
group1|test001,test002
test003|test003
"""

ghl.import_assessment_groups_csv(name_assessment='a1', csv_input=IMPORT_STUDENT_GROUPS_CSV)

## Prepare assessment
Creates a repository for each student group which will contain all of the assessment files as listed in the `main-dir` path (specified in `admin_course.ipynb`). The repositories will not be visible to students yet.
<br />
<br />
You must add at least a `README.md` to the `main-dir` path before calling this method.

* Creates student repositories visible only to instructors.

In [ ]:
ghl.prepare_assessment(name_assessment='a1', name_target_branch='master', overwrite=False)

## Release assessment
Makes the repositories visible to each student group by inviting them to their assessment repository group.<br />
<br />
Note that a FAIL message will be displayed for each student who has not yet checked their e-mail and accepted the invitation to join the organisation. <br />

Available student permissions:
* push = read and write
* pull = read only
* admin = full control

In [ ]:
ghl.release_assessment(name_assessment='a1', permission='push')

## Update assessment
Pushes any changes made to the assessment files located in the instructors directory under `main-dir` (specified in `admin_course.ipynb`). <br />
<br />
Modified assessment files will be pushed to the `repo_update_branch` branch specified in the local course configuration JSON file. Each student in the group will recieve a pull request to accept the changes.

* Calls release_assessment to transfer new files to the instructor_updates branch for each students assessment repository.
* Creates a pull request notifying the student to merge the instructor_updates branch with master.

In [ ]:
ghl.update_assessment_pr(name_assessment='a1')

## Close assessment
Repositories will not automatically be set to read only once the due date has passed. This method will seek the `deadline` value specified in `admin_course.ipynb` for the given assessment and find the most recent commit to `master` before the deadline.<br />
<br />
Creates a ZIP archive of the most recent commit and stores it in the instructors repository.

* Sets student access to pull (read) on their repository.
* Stores a ZIP copy of their submission.

In [ ]:
ghl.close_assessment(name_assessment='a1', compress=True)

## Forfeit assessment
To be used to release an item of assessment back to the user. Helps in reducing the amount of repositories in the organisation.

* Assigns admin rights to each member of the repository.
* Messages the users to request they transfer ownership to themselves.

In [ ]:
ghl.forfeit_assessment('a1')